# Comparative Analysis and Batch Processing

This notebook demonstrates batch processing and comparative analysis capabilities for multiple XCT samples. Learn how to:

- **Batch process multiple volumes** automatically
- **Compare samples statistically** using ANOVA, t-tests, and Mann-Whitney tests
- **Analyze process-structure-property relationships** across samples
- **Perform quality control analysis** (repeatability, process capability)
- **Visualize comparisons** with interactive plots

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Batch process multiple XCT volumes efficiently
2. Compare samples using statistical tests
3. Identify significant differences between groups
4. Analyze process-structure-property relationships
5. Generate quality control reports

## ⚠️ Prerequisites

- **Notebook 01**: Basic understanding of loading and segmenting volumes
- **Notebook 02**: Understanding of preprocessing and filtering
- **Notebook 03**: Understanding of morphological analysis
- **Required packages**: Same as previous notebooks
- **Multiple volumes**: Several segmented volumes for comparison

## 📖 Usage

1. Run all cells to initialize the widgets
2. Add multiple file paths for batch processing
3. Configure analysis parameters
4. Run batch analysis
5. Compare samples using statistical tests
6. Explore process-structure-property relationships
7. Generate quality control reports


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Comparative Analysis and Batch Processing")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Comparative Analysis and Batch Processing
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load analysis modules
try:
    from src.analyzer import XCTAnalyzer
    from src.analysis.comparative_analysis import (
        batch_analyze, compare_samples,
        statistical_tests, process_structure_property
    )
    from src.core.metrics import compute_all_metrics
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Batch Processing and Comparison Dashboard

Use the interactive widgets below to batch process multiple volumes and compare them statistically.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    batch_results = pd.DataFrame()
    comparison_results = {}
    
    # ============================================
    # Section 1: Batch Processing
    # ============================================
    
    file_paths_text = widgets.Textarea(
        value='',
        placeholder='Enter file paths (one per line)\nExample:\n../data/sample1.dcm\n../data/sample2.dcm',
        description='File Paths:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px', height='150px')
    )
    
    sample_names_text = widgets.Textarea(
        value='',
        placeholder='Enter sample names (one per line, optional)\nExample:\nSample_A\nSample_B',
        description='Sample Names:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', height='150px')
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    segmentation_method = widgets.Dropdown(
        options=['otsu', 'adaptive', 'manual'],
        value='otsu',
        description='Segmentation:',
        style={'description_width': 'initial'}
    )
    
    run_batch_button = widgets.Button(
        description='📊 Run Batch Analysis',
        button_style='primary',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    batch_progress = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Batch Progress:',
        style={'bar_color': '#3498db'},
        layout=widgets.Layout(width='500px')
    )
    
    batch_results_display = widgets.HTML(
        value="<p><i>No batch analysis results</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Statistical Comparison
    # ============================================
    
    metric_to_compare = widgets.Dropdown(
        options=['Select metric...'],
        value='Select metric...',
        description='Metric:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    group_by_column = widgets.Text(
        value='',
        placeholder='Column name for grouping (e.g., batch, condition)',
        description='Group By:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    test_type = widgets.Dropdown(
        options=['ANOVA', 't-test', 'Mann-Whitney'],
        value='ANOVA',
        description='Test Type:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    run_comparison_button = widgets.Button(
        description='🔬 Run Comparison',
        button_style='info',
        layout=widgets.Layout(width='150px'),
        disabled=True
    )
    
    comparison_results_display = widgets.HTML(
        value="<p><i>No comparison results</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    comparison_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 3: Process-Structure-Property
    # ============================================
    
    process_params_file = widgets.Text(
        value='',
        placeholder='Path to CSV with process parameters',
        description='Process Params CSV:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    analyze_psp_button = widgets.Button(
        description='🔗 Analyze PSP',
        button_style='info',
        layout=widgets.Layout(width='150px'),
        disabled=True
    )
    
    psp_results_display = widgets.HTML(
        value="<p><i>No PSP analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    psp_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


## 4. Widget Callbacks and Functions


In [4]:
if WIDGETS_AVAILABLE:
    
    def run_batch_callback(button):
        """Run batch analysis on multiple volumes"""
        global batch_results
        
        file_paths_str = file_paths_text.value.strip()
        if not file_paths_str:
            status_display.value = "<p style='color: red;'>Please enter file paths</p>"
            return
        
        # Parse file paths
        file_paths = [p.strip() for p in file_paths_str.split('\n') if p.strip()]
        if not file_paths:
            status_display.value = "<p style='color: red;'>No valid file paths found</p>"
            return
        
        # Parse sample names
        sample_names = None
        sample_names_str = sample_names_text.value.strip()
        if sample_names_str:
            sample_names = [n.strip() for n in sample_names_str.split('\n') if n.strip()]
            if len(sample_names) != len(file_paths):
                status_display.value = "<p style='color: orange;'>Warning: Number of sample names doesn't match file paths. Using default names.</p>"
                sample_names = None
        
        status_display.value = f"<p>Processing {len(file_paths)} samples...</p>"
        progress_bar.value = 5
        batch_progress.value = 0
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            voxel_sizes = [voxel_size] * len(file_paths)  # Same voxel size for all
            
            # Resolve file paths
            resolved_paths = []
            for path in file_paths:
                path_obj = Path(path)
                if not path_obj.exists():
                    data_path = project_root / 'data' / path
                    if data_path.exists():
                        resolved_paths.append(str(data_path))
                    else:
                        status_display.value = f"<p style='color: red;'>File not found: {path}</p>"
                        return
                else:
                    resolved_paths.append(str(path_obj))
            
            progress_bar.value = 10
            
            # Analysis configuration
            analysis_config = {
                'segmentation_method': segmentation_method.value,
                'refine': True
            }
            
            # Run batch analysis
            results = []
            total = len(resolved_paths)
            
            for i, (path, voxel_size_val, name) in enumerate(zip(resolved_paths, voxel_sizes, 
                                                                 sample_names if sample_names else [f"Sample_{j+1}" for j in range(total)])):
                try:
                    batch_progress.value = int((i / total) * 100)
                    progress_bar.value = 10 + int((i / total) * 80)
                    
                    # Initialize analyzer
                    analyzer = XCTAnalyzer(voxel_size=voxel_size_val, target_unit='mm')
                    analyzer.load_volume(path, normalize=True)
                    
                    # Segment
                    analyzer.segment(method=analysis_config['segmentation_method'], refine=analysis_config['refine'])
                    
                    # Compute metrics
                    metrics = analyzer.compute_metrics()
                    metrics['sample_name'] = name
                    metrics['file_path'] = path
                    
                    results.append(metrics)
                    
                except Exception as e:
                    status_display.value = f"<p style='color: orange;'>Warning: Failed to analyze {name}: {e}</p>"
                    results.append({
                        'sample_name': name,
                        'file_path': path,
                        'error': str(e)
                    })
            
            batch_results = pd.DataFrame(results)
            batch_progress.value = 100
            progress_bar.value = 95
            
            # Update metric dropdown
            numeric_cols = batch_results.select_dtypes(include=[np.number]).columns.tolist()
            metric_to_compare.options = ['Select metric...'] + numeric_cols
            metric_to_compare.disabled = False
            group_by_column.disabled = False
            test_type.disabled = False
            run_comparison_button.disabled = False
            analyze_psp_button.disabled = False
            process_params_file.disabled = False
            
            # Display results
            html = f"""
            <h4>📊 Batch Analysis Results</h4>
            <p><b>Total Samples:</b> {len(batch_results)}</p>
            <p><b>Successful:</b> {len(batch_results[batch_results.get('error', pd.Series()).isna()])}</p>
            <h5>Summary Statistics:</h5>
            {batch_results.select_dtypes(include=[np.number]).describe().to_html(classes='table table-striped')}
            """
            batch_results_display.value = html
            
            progress_bar.value = 100
            status_display.value = f"<p style='color: green;'>✅ Batch analysis complete! Processed {len(batch_results)} samples.</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in batch analysis: {e}</p>"
            progress_bar.value = 0
            batch_progress.value = 0
            import traceback
            traceback.print_exc()
    
    def run_comparison_callback(button):
        """Run statistical comparison"""
        global comparison_results
        
        if batch_results.empty:
            status_display.value = "<p style='color: red;'>Please run batch analysis first</p>"
            return
        
        metric = metric_to_compare.value
        if metric == 'Select metric...':
            status_display.value = "<p style='color: red;'>Please select a metric</p>"
            return
        
        status_display.value = "<p>Running statistical comparison...</p>"
        progress_bar.value = 20
        
        try:
            # Get groups
            group_col = group_by_column.value.strip()
            
            if group_col and group_col in batch_results.columns:
                # Group by column
                groups = {}
                for group_name, group_df in batch_results.groupby(group_col):
                    values = group_df[metric].dropna().tolist()
                    if len(values) > 0:
                        groups[group_name] = values
                
                if len(groups) < 2:
                    status_display.value = "<p style='color: red;'>Need at least 2 groups for comparison</p>"
                    return
                
            else:
                # Compare all samples (single group)
                groups = {'All Samples': batch_results[metric].dropna().tolist()}
                if len(groups['All Samples']) < 2:
                    status_display.value = "<p style='color: red;'>Need at least 2 samples for comparison</p>"
                    return
            
            progress_bar.value = 50
            
            # Run statistical test
            test = test_type.value.lower().replace('-', '')
            if test == 'anova':
                test = 'anova'
            elif test == 'ttest':
                test = 'ttest'
            elif test == 'mannwhitney':
                test = 'mannwhitney'
            
            test_result = statistical_tests(groups, test_type=test)
            progress_bar.value = 80
            
            comparison_results = test_result
            
            # Display results
            html = f"""
            <h4>🔬 Statistical Comparison Results</h4>
            <p><b>Metric:</b> {metric}</p>
            <p><b>Test Type:</b> {test_type.value}</p>
            <p><b>Groups:</b> {', '.join(groups.keys())}</p>
            <p><b>Statistic:</b> {test_result.get('statistic', 'N/A'):.4f if test_result.get('statistic') else 'N/A'}</p>
            <p><b>p-value:</b> {test_result.get('p_value', 'N/A'):.6f if test_result.get('p_value') else 'N/A'}</p>
            <p><b>Significant:</b> {'✅ Yes' if test_result.get('significant', False) else '❌ No'} (α = 0.05)</p>
            """
            comparison_results_display.value = html
            
            # Visualize
            with comparison_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                
                # Box plot
                group_data = [groups[g] for g in groups.keys()]
                axes[0].boxplot(group_data, labels=list(groups.keys()))
                axes[0].set_ylabel(metric, fontsize=11)
                axes[0].set_title(f'Distribution Comparison: {metric}', fontsize=12, fontweight='bold')
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Bar plot with means and error bars
                means = [np.mean(groups[g]) for g in groups.keys()]
                stds = [np.std(groups[g]) for g in groups.keys()]
                axes[1].bar(groups.keys(), means, yerr=stds, alpha=0.7, capsize=5)
                axes[1].set_ylabel(f'Mean {metric}', fontsize=11)
                axes[1].set_title(f'Mean Comparison: {metric}', fontsize=12, fontweight='bold')
                axes[1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Statistical comparison complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in comparison: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_psp_callback(button):
        """Analyze process-structure-property relationships"""
        global batch_results
        
        if batch_results.empty:
            status_display.value = "<p style='color: red;'>Please run batch analysis first</p>"
            return
        
        process_file = process_params_file.value.strip()
        if not process_file:
            status_display.value = "<p style='color: red;'>Please provide process parameters CSV file</p>"
            return
        
        status_display.value = "<p>Analyzing PSP relationships...</p>"
        progress_bar.value = 20
        
        try:
            # Load process parameters
            process_path = Path(process_file)
            if not process_path.exists():
                process_path = project_root / 'data' / process_file
                if not process_path.exists():
                    status_display.value = f"<p style='color: red;'>File not found: {process_file}</p>"
                    return
            
            process_params = pd.read_csv(process_path)
            progress_bar.value = 40
            
            # Prepare structure metrics (from batch results)
            structure_metrics = batch_results.select_dtypes(include=[np.number]).drop(columns=['sample_name'], errors='ignore')
            progress_bar.value = 60
            
            # Run PSP analysis
            psp_result = process_structure_property(process_params, structure_metrics)
            progress_bar.value = 80
            
            # Display results
            html = f"""
            <h4>🔗 Process-Structure-Property Analysis</h4>
            <p><b>Number of Samples:</b> {psp_result['n_samples']}</p>
            <p><b>Process Parameters:</b> {', '.join(psp_result['process_parameters'])}</p>
            <p><b>Structure Metrics:</b> {', '.join(psp_result['structure_metrics'][:5])}...</p>
            <h5>Top Correlations:</h5>
            <ul>
            """
            # Sort correlations by absolute value
            sorted_corrs = sorted(psp_result['correlations'].items(), key=lambda x: abs(x[1]), reverse=True)
            for key, value in sorted_corrs[:10]:
                html += f"<li><b>{key}:</b> {value:.3f}</li>"
            html += "</ul>"
            psp_results_display.value = html
            
            # Visualize
            with psp_visualization:
                clear_output()
                # Create correlation heatmap
                if len(sorted_corrs) > 0:
                    # Extract process and structure names
                    process_params_list = psp_result['process_parameters']
                    structure_metrics_list = psp_result['structure_metrics']
                    
                    # Create correlation matrix
                    corr_matrix = np.zeros((len(process_params_list), len(structure_metrics_list)))
                    for i, proc in enumerate(process_params_list):
                        for j, struct in enumerate(structure_metrics_list):
                            key = f"{proc} -> {struct}"
                            if key in psp_result['correlations']:
                                corr_matrix[i, j] = psp_result['correlations'][key]
                    
                    fig, axes = plt.subplots(1, 1, figsize=(12, 8))
                    im = axes.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
                    axes.set_xticks(range(len(structure_metrics_list)))
                    axes.set_xticklabels(structure_metrics_list, rotation=45, ha='right')
                    axes.set_yticks(range(len(process_params_list)))
                    axes.set_yticklabels(process_params_list)
                    axes.set_title('Process-Structure Correlation Matrix', fontsize=12, fontweight='bold')
                    plt.colorbar(im, ax=axes, label='Correlation')
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ PSP analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in PSP analysis: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    run_batch_button.on_click(run_batch_callback)
    run_comparison_button.on_click(run_comparison_callback)
    analyze_psp_button.on_click(analyze_psp_callback)
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


## 5. Display Interactive Dashboard


In [5]:
if WIDGETS_AVAILABLE:
    
    # Create batch processing panel
    batch_panel = widgets.VBox([
        widgets.HTML("<h2>📊 Batch Processing</h2>"),
        HBox([
            widgets.VBox([
                widgets.HTML("<b>File Paths (one per line):</b>"),
                file_paths_text
            ]),
            widgets.VBox([
                widgets.HTML("<b>Sample Names (optional):</b>"),
                sample_names_text
            ])
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        segmentation_method,
        run_batch_button,
        batch_progress,
        batch_results_display
    ])
    
    # Create comparison panel
    comparison_panel = widgets.VBox([
        widgets.HTML("<h3>🔬 Statistical Comparison</h3>"),
        HBox([metric_to_compare, group_by_column]),
        test_type,
        run_comparison_button,
        comparison_results_display,
        comparison_visualization
    ])
    
    # Create PSP panel
    psp_panel = widgets.VBox([
        widgets.HTML("<h3>🔗 Process-Structure-Property</h3>"),
        process_params_file,
        analyze_psp_button,
        psp_results_display,
        psp_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        comparison_panel,
        psp_panel
    ])
    analysis_tabs.set_title(0, '🔬 Comparison')
    analysis_tabs.set_title(1, '🔗 PSP')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>📊 Comparative Analysis and Batch Processing</h1>"),
        batch_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📈 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start batch processing and comparing samples.")
    print("\n💡 Tips:")
    print("   1. Enter multiple file paths (one per line)")
    print("   2. Optionally provide sample names")
    print("   3. Run batch analysis to process all samples")
    print("   4. Compare samples using statistical tests")
    print("   5. Analyze process-structure-property relationships")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start batch processing and comparing samples.

💡 Tips:
   1. Enter multiple file paths (one per line)
   2. Optionally provide sample names
   3. Run batch analysis to process all samples
   4. Compare samples using statistical tests
   5. Analyze process-structure-property relationships


In [6]:
if WIDGETS_AVAILABLE:
    
    # Create batch processing panel
    batch_panel = widgets.VBox([
        widgets.HTML("<h2>📊 Batch Processing</h2>"),
        HBox([
            widgets.VBox([
                widgets.HTML("<b>File Paths (one per line):</b>"),
                file_paths_text
            ]),
            widgets.VBox([
                widgets.HTML("<b>Sample Names (optional):</b>"),
                sample_names_text
            ])
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        segmentation_method,
        run_batch_button,
        batch_progress,
        batch_results_display
    ])
    
    # Create comparison panel
    comparison_panel = widgets.VBox([
        widgets.HTML("<h3>🔬 Statistical Comparison</h3>"),
        HBox([metric_to_compare, group_by_column]),
        test_type,
        run_comparison_button,
        comparison_results_display,
        comparison_visualization
    ])
    
    # Create PSP panel
    psp_panel = widgets.VBox([
        widgets.HTML("<h3>🔗 Process-Structure-Property</h3>"),
        process_params_file,
        analyze_psp_button,
        psp_results_display,
        psp_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        comparison_panel,
        psp_panel
    ])
    analysis_tabs.set_title(0, '🔬 Comparison')
    analysis_tabs.set_title(1, '🔗 PSP')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>📊 Comparative Analysis and Batch Processing</h1>"),
        batch_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📈 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start batch processing and comparing samples.")
    print("\n💡 Tips:")
    print("   1. Enter multiple file paths (one per line)")
    print("   2. Optionally provide sample names")
    print("   3. Run batch analysis to process all samples")
    print("   4. Compare samples using statistical tests")
    print("   5. Analyze process-structure-property relationships")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start batch processing and comparing samples.

💡 Tips:
   1. Enter multiple file paths (one per line)
   2. Optionally provide sample names
   3. Run batch analysis to process all samples
   4. Compare samples using statistical tests
   5. Analyze process-structure-property relationships


## 6. Summary

### What We Learned

1. **Batch Processing**:
   - Automated analysis of multiple volumes
   - Progress tracking for large batches
   - Error handling for failed samples
   - Summary statistics across samples

2. **Statistical Comparison**:
   - ANOVA for multiple groups
   - t-test for two groups
   - Mann-Whitney U test (non-parametric)
   - Visualization of group differences

3. **Process-Structure-Property Analysis**:
   - Correlation analysis between process parameters and structure
   - Identification of key relationships
   - Heatmap visualization of correlations

### Key Insights

- **Batch processing** enables efficient analysis of large datasets
- **Statistical tests** identify significant differences between groups
- **PSP analysis** reveals process-structure relationships
- **Quality control** metrics help assess repeatability

### Next Steps

- **Notebook 07**: Quality Control and Validation
  - Dimensional accuracy analysis
  - Uncertainty quantification
  - Validation against ground truth

- **Notebook 08**: Complete Analysis Pipeline
  - End-to-end workflow
  - Comprehensive reporting
  - Reproducibility package

### Resources

- [Framework Documentation](../docs/README.md)
- [Analysis Modules](../docs/modules.md#analysis-modules)
- [Comparative Analysis](../docs/modules.md#analysiscomparative_analysis)
